# Import the necessary libraries

In [1]:
!pip uninstall -y spacy

Found existing installation: spacy 2.3.9
Uninstalling spacy-2.3.9:
  Successfully uninstalled spacy-2.3.9


In [2]:
!python -m pip install spacy~=2.0

  Using cached spacy-2.3.9-cp39-cp39-win_amd64.whl (9.1 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.2.2 requires spacy<4.0.0,>=3.5.0, but you have spacy 2.3.9 which is incompatible.
spacy-transformers 1.2.2 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.6 which is incompatible.
en-core-web-md 3.3.0 requires spacy<3.4.0,>=3.3.0.dev0, but you have spacy 2.3.9 which is incompatible.


In [6]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aiohttp                       3.8.1
aiosignal                     1.3.1
alabaster                     0.7.12
altair                        4.2.2
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     3.0.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backp

In [7]:
!pip uninstall -y spacy-transformers

Found existing installation: spacy-transformers 1.2.2
Uninstalling spacy-transformers-1.2.2:
  Successfully uninstalled spacy-transformers-1.2.2


In [6]:
import spacy
import pickle
import random


# Load the training data

In [7]:
train_data = pickle.load(open('C:/Users/KIIT/Minor_project/ResumeParsing code/train_data.pkl', 'rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [8]:
nlp = spacy.blank('en')
def train_model(train_data, test_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)

    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(1000):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorize data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass

            # Compute accuracy on test dataset
            correct = 0
            total = 0
            for text, annotations in test_data:
                doc = nlp(text)
                for ent in doc.ents:
                    if (ent.start_char, ent.end_char, ent.label_) in annotations['entities']:
                        correct += 1
                    total += 1
            accuracy = correct / total
            print(f"Accuracy after iteration {itn}: {accuracy:.3f}")


In [9]:
train_model(train_data, train_data)

Starting iteration 0
Accuracy after iteration 0: 0.552
Starting iteration 1
Accuracy after iteration 1: 0.579
Starting iteration 2
Accuracy after iteration 2: 0.492
Starting iteration 3
Accuracy after iteration 3: 0.477
Starting iteration 4
Accuracy after iteration 4: 0.586
Starting iteration 5
Accuracy after iteration 5: 0.521
Starting iteration 6
Accuracy after iteration 6: 0.486
Starting iteration 7
Accuracy after iteration 7: 0.609
Starting iteration 8
Accuracy after iteration 8: 0.590
Starting iteration 9
Accuracy after iteration 9: 0.560


# Save the model for further use

In [10]:
nlp.to_disk('nlp_model')

# Load the saved model into nlp_model

In [11]:
nlp_model = spacy.load('nlp_model')

# First resume from our training data

In [12]:
train_data[0]

('Pavithra M "Infosys" internship  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Pavithra-M/26f392ec8251143b  Willing to relocate to: Bengaluru, Karnataka  WORK EXPERIENCE  "Infosys" internship  Infosys  About: a detailed analysis of diseases and user interface based on that giving clear idea which diseases has affected at what region and treatment availability for that particular diseases. It was completed in "Infosys" internship • Event management. • Good communication skills. • Leadership qualities. • Team collaboration. • Creative thinking.  EDUCATION  CBSE  STATE  SSLC  State board  SKILLS  ADOBE PHOTOSHOP (Less than 1 year), ANDROID (Less than 1 year), APPLICATION SOFTWARE (Less than 1 year), C++ (Less than 1 year), CSS (Less than 1 year)  ADDITIONAL INFORMATION  Technical skills:  • Programing Languages: C, C++ and JAVA.  • Databases: MySQL, SQL server (SQLyog)  • Web tools: HTML, CSS.  • Operating Systems: Windows (XP, 8, 8.1, 10), Ubuntu, fedora, Linux.  https://www.

# Pass the resume to the model and see results

In [13]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Pavithra M
COMPANIES WORKED AT           - Infosys
LOCATION                      - Bengaluru
EMAIL ADDRESS                 - indeed.com/r/Pavithra-M/26f392ec8251143b
LOCATION                      - Bengaluru
COMPANIES WORKED AT           - Infosys
DESIGNATION                   - internship
COMPANIES WORKED AT           - Infosys
COMPANIES WORKED AT           - Infosys
SKILLS                        - ADOBE PHOTOSHOP (Less than 1 year), ANDROID (Less than 1 year), APPLICATION SOFTWARE (Less than 1 year), C++ (Less than 1 year), CSS (Less than 1 year)


In [13]:
!pip install PyMuPDF

In [14]:
import sys, fitz
fname = 'Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for Page in doc:
    text = text + str(Page.get_text())

tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [15]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
LOCATION                      - Delhi
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Technology/Tools used
COLLEGE NAME                  - Indian Institute of Technology – Mumbai
GRADUATION YEAR               - 2001
